In [1]:
import torch
from transformers import BertForSequenceClassification, ElectraForSequenceClassification, BertTokenizer, ElectraTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_from_disk
import nltk
from textattack.attack_recipes import TextFoolerJin2019
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack import Attacker
from sklearn.metrics import accuracy_score, f1_score

In [2]:
dataset = load_dataset("imdb")

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

In [3]:
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [3]:
try:
    bert_encoded = load_from_disk(".encoded_datasets/bert_encoded")
    print("Loaded BERT-encoded dataset from disk.")
except:
    # If not saved, tokenize and save
    bert_encoded = dataset.map(lambda x: tokenize_function(x, bert_tokenizer), batched=True)
    bert_encoded.save_to_disk(".encoded_datasets/bert_encoded")
    print("BERT-encoded dataset saved to disk.")

# Check if the ELECTRA-encoded dataset is already saved
try:
    electra_encoded = load_from_disk(".encoded_datasets/electra_encoded")
    print("Loaded ELECTRA-encoded dataset from disk.")
except:
    # If not saved, tokenize and save
    electra_encoded = dataset.map(lambda x: tokenize_function(x, electra_tokenizer), batched=True)
    electra_encoded.save_to_disk(".encoded_datasets/electra_encoded")
    print("ELECTRA-encoded dataset saved to disk.")

torch.save

Loaded BERT-encoded dataset from disk.
Loaded ELECTRA-encoded dataset from disk.


<function torch.serialization.save(obj: object, f: Union[str, os.PathLike, BinaryIO, IO[bytes]], pickle_module: Any = <module 'pickle' from '/Users/awang/anaconda3/lib/python3.11/pickle.py'>, pickle_protocol: int = 2, _use_new_zipfile_serialization: bool = True, _disable_byteorder_record: bool = False) -> None>

In [6]:
# Load pre-trained models for classification
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
electra_model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=2)

# Define adversarial attack (TextFooler)
#attack = TextFoolerJin2019.build(bert_model)

#Adversarial Training Section

# Wrap dataset in TextAttack format
wrapped_model = HuggingFaceModelWrapper(bert_model, bert_tokenizer)

# Create the attack
attack = TextFoolerJin2019.build(wrapped_model)   
attack.max_candidates = 10   # Increase the number of candidates
attack.max_num_words = 5     # Allow more words to be perturbed
attack.threshold = 0.3       # Lower the threshold for more perturbation

# Use Hugging Face dataset (IMDB)
huggingface_dataset = HuggingFaceDataset("imdb", split='test')

# Create the attacker
attacker = Attacker(attack, huggingface_dataset)

# Run the attack and generate adversarial examples
results = attacker.attack_dataset()

# Display a few adversarial examples
for i in range(5):
    print("Original:", results[i].original_text())
    print("Adversarial:", results[i].perturbed_text())
    print("-" * 50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split test.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

--------------------------------------------- Result 1 ---------------------------------------------


[[Negative (61%)]] --> [[Positive (50%)]]

I [[love]] sci-fi and am willing to put up with a lot. Sci-fi movies/[[TV]] are [[usually]] underfunded, under-appreciated and [[misunderstood]]. I [[tried]] to like this, [[I]] [[really]] [[did]], but it is to [[good]] [[TV]] sci-fi as Babylon 5 is to [[Star]] [[Trek]] (the [[original]]). [[Silly]] [[prosthetics]], cheap cardboard [[sets]], [[stilted]] [[dialogues]], [[CG]] that doesn't [[match]] the [[background]], and [[painfully]] one-dimensional characters cannot [[be]] [[overcome]] with a 'sci-fi' [[setting]]. (I'm [[sure]] there are those of you out there who [[think]] [[Babylon]] 5 is [[good]] sci-fi [[TV]]. It's not. It's [[clichéd]] and [[uninspiring]].) [[While]] [[US]] [[viewers]] might like emotion and [[character]] [[development]], sci-fi is a [[genre]] that [[does]] not [[take]] itself [[seriously]] ([[cf]]. Star Trek). It may [[treat]] [[important]] [[issues]], yet not as a [[serious]] [[philosophy]]. It's [[really]] [[difficul

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (60%)]] --> [[Positive (50%)]]

Worth the [[entertainment]] [[value]] of a rental, especially if you [[like]] [[action]] [[movies]]. [[This]] one [[features]] the [[usual]] [[car]] [[chases]], [[fights]] with the [[great]] Van Damme [[kick]] [[style]], [[shooting]] [[battles]] with the 40 [[shell]] [[load]] [[shotgun]], and even [[terrorist]] [[style]] bombs. [[All]] of this is [[entertaining]] and competently [[handled]] but there is nothing that [[really]] [[blows]] you [[away]] if you've [[seen]] your [[share]] before.<br /><[[br]] />The [[plot]] is [[made]] [[interesting]] by the [[inclusion]] of a rabbit, which is [[clever]] but [[hardly]] profound. [[Many]] of the characters are [[heavily]] [[stereotyped]] -- the [[angry]] [[veterans]], the [[terrified]] [[illegal]] [[aliens]], the [[crooked]] [[cops]], the [[indifferent]] [[feds]], the [[bitchy]] [[tough]] [[lady]] [[s

...

--------------------------------------------- Result 7 ---------------------------------------------
[[Negative (63%)]] --> [[Positive (50%)]]

[[Isaac]] Florentine has [[made]] some of the [[best]] [[western]] Martial [[Arts]] [[action]] [[movies]] ever [[produced]]. [[In]] [[particular]] US Seals 2, [[Cold]] [[Harvest]], [[Special]] [[Forces]] and [[Undisputed]] 2 are all [[action]] classics. You can [[tell]] Isaac [[has]] a [[real]] [[passion]] for the [[genre]] and his [[films]] are [[always]] eventful, [[creative]] and [[sharp]] [[affairs]], with some of the [[best]] [[fight]] [[sequences]] an [[action]] fan could [[hope]] for. In [[particular]] he [[has]] [[found]] a muse with Scott Adkins, as [[talented]] an [[actor]] and [[action]] [[performer]] as you could [[hope]] for. [[This]] is [[borne]] out with Special Forces and [[Undisputed]] 2, but unfortunately The Shepherd just doesn't [[live]] up to their [[abilities]].<[[br]] /><[[br]] />There is no [[doubt]] that JCVD looks bett

--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (63%)]] --> [[Positive (50%)]]

[[It]] [[actually]] [[pains]] me to say it, but this [[movie]] was [[horrible]] on [[every]] [[level]]. The [[blame]] [[does]] not [[lie]] [[entirely]] with [[Van]] Damme as you can [[see]] he [[tried]] his [[best]], but let's [[face]] it, he's almost [[fifty]], how [[much]] more can you [[ask]] of him? [[I]] [[find]] it [[so]] [[hard]] to [[believe]] that the same [[people]] who [[put]] [[together]] Undisputed 2; [[arguably]] the [[best]] ([[western]]) martial [[arts]] [[movie]] in [[years]], [[created]] this. [[Everything]] from the [[plot]], to the dialog, to the [[editing]], to the [[overall]] [[acting]] was just [[horribly]] [[put]] [[together]] and in [[many]] [[cases]] outright [[boring]] and [[nonsensical]]. Scott Adkins who's fight [[scenes]] [[seemed]] more [[like]] a [[demo]] [[reel]], was also [[terribly]] underused and not even the

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (61%)]] --> [[Positive (50%)]]

[[Technically]] I'am a [[Van]] [[Damme]] [[Fan]], or [[I]] was. this [[movie]] is [[so]] [[bad]] that [[I]] [[hated]] myself for [[wasting]] those 90 [[minutes]]. [[Do]] not [[let]] the [[name]] Isaac Florentine (Undisputed [[II]]) fool you, [[I]] [[had]] [[big]] [[hopes]] for this one, [[depending]] on what [[I]] [[saw]] in ([[Undisputed]] [[II]]), [[man]].. was [[I]] [[wrong]] ??! all [[action]] [[fans]] [[wanted]] a [[big]] [[comeback]] for the [[classic]] [[action]] [[hero]], but i guess we [[wont]] [[be]] [[able]] to [[see]] that [[soon]], as our [[hero]] [[keep]] [[coming]] with those ([[going]] -to-a-border - far-away-town-and -[[kill]] -the-bad-guys- than-comeback- [[home]]) [[movies]] I [[mean]] for God's [[sake]], we are in 2008, and they [[insist]] on [[doing]] those disappointing [[movies]] on [[every]] [[level]]. [[Why]] ??!!! [[Do

[Succeeded / Failed / Skipped / Total] 9 / 1 / 0 / 10: 100%|██████████| 10/10 [41:32<00:00, 249.24s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (60%)]] --> [[[FAILED]]]

Honestly awful film, bad editing, awful lighting, dire dialog and scrappy screenplay.<br /><br />The lighting at is so bad there's moments you can't even see what's going on, I even tried to playing with the contrast and brightness so I could see something but that didn't help.<br /><br />They must have found the script in a bin, the character development is just as awful and while you hardly expect much from a Jean-Claude Van Damme film this one manages to hit an all time low. You can't even laugh at the cheesy'ness.<br /><br />The directing and editing are also terrible, the whole film follows an extremely tired routine and fails at every turn as it bumbles through the plot that is so weak it's just unreal.<br /><br />There's not a lot else to say other than it's really bad and nothing like Jean-Claude Van Damme's earlier work which you could enjo

In [6]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
)

try:
    bert_model = BertForSequenceClassification.from_pretrained(".trained_models/bert_model")
    print("Loaded trained BERT model from disk.")
except:
    # If not saved, tokenize and save
    trainer_bert = Trainer(
    model=bert_model,  # Fine-tune BERT model
    args=training_args,
    train_dataset=bert_encoded['train'],
    eval_dataset=bert_encoded['test'],
    )

    trainer_bert.train()
    trainer_bert.save_model(".trained_models/bert_model")
    print("BERT model saved to disk.")
# # Check if the ELECTRA model is already saved
# try:
#     electra_model = ElectraForSequenceClassification.from_pretrained("./test/saved_model/")

#     print("Loaded trained ELECTRA model from disk.")
# except:
#     # If not saved, tokenize and save
#     trainer_electra = Trainer(
#     model=electra_model,  # Fine-tune Electra model
#     args=training_args,
#     train_dataset=electra_encoded['train'],
#     eval_dataset=electra_encoded['test'],
#     )

#     # Fine-tune Electra model
#     trainer_electra.train()
#     trainer_electra.save_model(".trained_models/electra_model")
#     print("ELECTRA model saved to disk.")


# Evaluate the fine-tuned models


trainer_bert = Trainer(
    model=bert_model,  # Fine-tune BERT model
    args=training_args,
    train_dataset=bert_encoded['train'],
    eval_dataset=bert_encoded['test'],
    )

bert_eval = trainer_bert.evaluate()
# electra_eval = trainer_electra.evaluate()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/awang/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Loaded trained BERT model from disk.


  0%|          | 0/3125 [00:00<?, ?it/s]

In [7]:
# Adversarial Attack Section
bert_model = BertForSequenceClassification.from_pretrained(".trained_models/bert_model")

# Wrap dataset in TextAttack format
wrapped_model = HuggingFaceModelWrapper(bert_model, bert_tokenizer)

# Create the attack
attack = TextFoolerJin2019.build(wrapped_model)   
attack.max_candidates = 10   # Increase the number of candidates
attack.max_num_words = 5     # Allow more words to be perturbed
attack.threshold = 0.3       # Lower the threshold for more perturbation

# Use Hugging Face dataset (IMDB)
huggingface_dataset = HuggingFaceDataset("imdb", split='test')

# Create the attacker
attacker = Attacker(attack, huggingface_dataset)

# Run the attack and generate adversarial examples
results = attacker.attack_dataset()

# Display a few adversarial examples
for i in range(5):
    print("Original:", results[i].original_text())
    print("Adversarial:", results[i].perturbed_text())
    print("-" * 50)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split test.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:32<04:49, 32.20s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (97%)]]

I [[love]] sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, [[stilted]] [[dialogues]], CG that doesn't match the background, and painfully one-dimensional [[characters]] cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who [[think]] Babylon 5 is [[good]] sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not [[take]] itself seriously (cf. Star Trek). It may treat important issues, yet not as a [[serious]] philosophy. It's really [[difficult]] to [[care]] about the [[characters]] here as they are 

[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:51<03:27, 25.96s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (94%)]]

It actually [[pains]] me to say it, but this movie was [[horrible]] on every level. The blame does not lie entirely with Van Damme as you can see he [[tried]] his best, but let's face it, he's almost fifty, how much more can you ask of him? I find it so hard to believe that the same people who put together Undisputed 2; arguably the best (western) martial [[arts]] movie in years, created this. Everything from the [[plot]], to the dialog, to the editing, to the overall [[acting]] was just [[horribly]] put together and in many cases outright [[boring]] and [[nonsensical]]. Scott Adkins who's fight scenes seemed more like a demo reel, was also terribly underused and not even the main villain which is such a shame because 1) He is more than capable of playing that role and 2) The actual main villain was not only not intimidating at all but also qu

[Succeeded / Failed / Skipped / Total] 8 / 0 / 1 / 9:  90%|█████████ | 9/10 [02:51<00:19, 19.04s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (88%)]]

Technically I'am a Van Damme Fan, or I was. this [[movie]] is [[so]] [[bad]] that I [[hated]] myself for [[wasting]] those 90 [[minutes]]. Do not let the name Isaac Florentine (Undisputed II) fool you, I had big hopes for this one, depending on what I saw in (Undisputed II), man.. was I wrong ??! all action fans [[wanted]] a big comeback for the classic action [[hero]], but i guess we wont be able to see that soon, as our hero keep coming with those (going -to-a-border - far-away-town-and -kill -the-bad-guys- than-comeback- home) [[movies]] I mean for God's sake, we are in 2008, and they insist on [[doing]] those [[disappointing]] [[movies]] on [[every]] [[level]]. Why ??!!! Do your self a favor, [[skip]] it.. [[seriously]].

Technically I'am a Van Damme Fan, or I was. this [[cinemas]] is [[too]] [[wicked]] that I [[dislikes]] myself for [[los

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10: 100%|██████████| 10/10 [03:24<00:00, 20.42s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (72%)]]

Honestly [[awful]] film, bad editing, awful lighting, dire dialog and [[scrappy]] screenplay.<br /><br />The lighting at is so bad there's moments you can't even see what's going on, I even tried to playing with the contrast and brightness so I could see something but that didn't [[help]].<br /><br />They must have found the script in a bin, the character development is just as [[awful]] and while you hardly expect [[much]] from a Jean-Claude Van Damme film this one manages to hit an all time low. You can't even laugh at the cheesy'ness.<br /><br />The [[directing]] and [[editing]] are also [[terrible]], the whole [[film]] [[follows]] an [[extremely]] tired routine and [[fails]] at every turn as it bumbles through the [[plot]] that is [[so]] weak it's just [[unreal]].<br /><br />There's not a lot else to [[say]] other than it's [[really]] bad

## Synonym Replacement on Attacking Data

In [ ]:
nltk.download('wordnet')

def synonym_replacement(sentence):
    words = sentence.split()
    new_sentence = []
    for word in words:
        synonyms = nltk.corpus.wordnet.synsets(word)
        if synonyms:
            i = 0
            while synonyms[i].lemmas()[0].name() == word:
                i += 1
                if i >= len(synonyms) - 1:
                    break
            if i >= len(synonyms) - 1:
                synonym = word
            else:
                synonym = synonyms[i].lemmas()[0].name()
            new_sentence.append(synonym)
        else:
            new_sentence.append(word)
    return ' '.join(new_sentence)

# Apply synonym replacement to the entire dataset
def synonym_replacement_dataset(dataset):
    dataset_synonym_replacement = dataset.map(lambda x: {'text': synonym_replacement(x['text'])})
    return dataset_synonym_replacement

[nltk_data] Downloading package wordnet to /Users/awang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
# Example of input transformation
sentence = "The quick brown fox jumps over the lazy dog."
transformed_sentence = synonym_replacement(sentence)
print("Original sentence:", sentence)
print("Transformed sentence:", transformed_sentence)

Original sentence: The quick brown fox jumps over the lazy dog.
Transformed sentence: The flying Brown dodger jump complete the faineant dog.


In [25]:
results[0].perturbed_result.ground_truth_output

0

In [27]:
original_synonym_replacement = [None] * 10
for i in range(10):
    original_synonym_replacement[i] = synonym_replacement(results[i].original_text())

original_texts = []
adversarial_texts = []
labels = []

#attack the model using the synonym replacement examples
for i in range(len(original_synonym_replacement)):
    text = original_synonym_replacement[i]
    label = results[i].perturbed_result.ground_truth_output

    try:
        # Generate adversarial example
        result = attack.attack(text, label)
        original_texts.append(result.original_text())
        adversarial_texts.append(result.perturbed_text())
        labels.append(label)
    except Exception as e:
        print(f"Error on sample {i}: {e}")
        continue

IndexError: list index out of range

In [ ]:
wrapped_model = HuggingFaceModelWrapper(bert_model, bert_tokenizer)

# Create the attack
attack = TextFoolerJin2019.build(wrapped_model)   
attack.max_candidates = 10   # Increase the number of candidates
attack.max_num_words = 5     # Allow more words to be perturbed
attack.threshold = 0.3       # Lower the threshold for more perturbation

# Use Hugging Face dataset (IMDB)
huggingface_dataset = HuggingFaceDataset("imdb", split='test')


# Apply synonym replacement to the attack dataset
attack_synonym_replacement = synonym_replacement_dataset(huggingface_dataset)

attacker = Attacker(attack, attack_synonym_replacement)

# Run the attack and generate adversarial examples
results = attacker.attack_dataset()

# Display a few adversarial examples
for i in range(5):
    print("Original:", results[i].original_text())
    print("Adversarial:", results[i].perturbed_text())
    print("-" * 50)

AttributeError: 'HuggingFaceDataset' object has no attribute 'map'

In [ ]:
# Majority voting for ensemble methods
def majority_voting(predictions_bert, predictions_electra):
    votes = torch.stack([predictions_bert, predictions_electra])
    majority_vote = torch.mode(votes, dim=0)[0]
    return majority_vote

preds_bert = trainer_bert.predict(bert_encoded['test']).predictions
preds_electra = trainer_electra.predict(electra_encoded['test']).predictions

# Get majority vote predictions
final_predictions = majority_voting(preds_bert, preds_electra)

# Get majority vote predictions on the data in the attack dataset
attack_preds_bert = trainer_bert.predict(attack_synonym_replacement).predictions
attack_preds_electra = trainer_electra.predict(attack_synonym_replacement).predictions


# Evaluate using majority vote predictions
true_labels = bert_encoded['test']['label']
acc = accuracy_score(true_labels, final_predictions)
f1 = f1_score(true_labels, final_predictions, average='weighted')

print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")



In [ ]:
# generate a training dataset with adversarial examples
# Wrap dataset in TextAttack format
wrapped_model = HuggingFaceModelWrapper(bert_model, bert_tokenizer)

# Create the attack
attack = TextFoolerJin2019.build(wrapped_model)
attack.max_candidates = 10   # Increase the number of candidates
attack.max_num_words = 5     # Allow more words to be perturbed
attack.threshold = 0.3       # Lower the threshold for more perturbation

# Use Hugging Face dataset (IMDB)
huggingface_dataset = HuggingFaceDataset("imdb", split='train')

# Create the attacker
attacker = Attacker(attack, huggingface_dataset)




In [ ]:
def adversarial_training(model, tokenizer, dataset, adversarial_examples):
    # Augment dataset with adversarial examples and fine-tune model
    augmented_dataset = dataset.add_adversarial_examples(adversarial_examples)
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir="./adversarial_results",
            evaluation_strategy="epoch",
            per_device_train_batch_size=16,
            num_train_epochs=3
        ),
        train_dataset=augmented_dataset
    )
    trainer.train()


bert_model.save_pretrained('./models/bert_adversarial_trained')
bert_tokenizer.save_pretrained('./models/bert_adversarial_trained')

# Week 9-10: Evaluate Defense Strategies
def evaluate_defense_strategies(model, dataset, adversarial_examples, transformations):
    # Full-scale experiment comparing strategies
    base_metrics = evaluate_model(model, tokenizer, dataset)
    adversarial_metrics = evaluate_model(model, tokenizer, adversarial_examples)
    transformed_metrics = evaluate_model(model, tokenizer, transformations)
    return base_metrics, adversarial_metrics, transformed_metrics

# Week 11-12: Compile Results and Prepare Presentation
def compile_results(results):
    # Save and present results
    with open("final_results.txt", "w") as f:
        f.write(str(results))

# Main Pipeline
if __name__ == "__main__":
    # Load datasets
    train_data, test_data = load_datasets()

    # Initialize models
    tokenizer, model = initialize_model("bert-base-uncased")

    # Baseline evaluation
    baseline_metrics = evaluate_model(model, tokenizer, test_data)

    # Generate adversarial examples
    adversarial_examples = generate_adversarial_examples(model, tokenizer, test_data)

    # Adversarial training
    adversarial_training(model, tokenizer, train_data, adversarial_examples)

    # Input transformations
    transformed_examples = [input_transformation(example) for example in adversarial_examples]

    # Evaluate defense strategies
    results = evaluate_defense_strategies(model, test_data, adversarial_examples, transformed_examples)

    # Compile results
    compile_results(results)
